# GAN Evaluation - Inception Score (IS)

## Basic [Concept](https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/)

In [1]:
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random

from scipy.linalg import sqrtm

In [9]:
def calculate_fid_score(fvector1, fvector2):
    # calculate mean and covariance statistics
    mean1, sigma1 = fvector1.mean(axis=0), cov(fvector1, rowvar=False)
    mean2, sigma2 = fvector2.mean(axis=0), cov(fvector2, rowvar=False)
    
    # calculate sum squared difference between two mean vectors
    ssdiff = np.sum((mean1 - mean2) ** 2.0)
    
    # calculate sqrt of product between two covariance metrics
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # some elements in the resulting matrix may be imaginary
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [10]:
# define two collections of activations
fvector1 = random(10*2048)
fvector1 = fvector1.reshape((10,2048))
fvector2 = random(10*2048)
fvector2 = fvector2.reshape((10,2048))

In [11]:
# fid between act1 and act1
fid = calculate_fid_score(fvector1, fvector1)
print('FID (identical feature vectors): %.3f' % fid)

FID (identical feature vectors): -0.000


In [13]:
# fid between act1 and act2
fid = calculate_fid_score(fvector1, fvector2)
print('FID (different feature vectors): %.3f' % fid)

FID (different feature vectors): 354.907


## FID in Pytorch

In [5]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

In [6]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

In [10]:
def calculate_inception_score(images, n_split=1, eps=1E-16, batch_size=32, resize=False, cuda=False):
    
    N = len(images)
    
    assert batch_size > 0
    assert N > batch_size
    
    # set up dtype for inception v3 model
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor
        
    # set up dataloader
    data_loader = torch.utils.data.DataLoader(images, batch_size=batch_size)
    
    # load inception model
    print("Loading Inception V3 model...")
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    
    up = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=False).type(dtype)
    def get_yhats(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    # get predictions
    print("Predicting class probabilities for images per batch...")
    yhats = np.zeros((N, 1000))

    for i, batch in enumerate(data_loader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        yhats[i*batch_size:i*batch_size + batch_size_i] = get_yhats(batchv)

    # compute the mean kl-div
    print("Computing the mean kl-div...")
    split_scores = []

    for k in range(n_split):
        
        # retrieve p(y|x)
        yhats_ = yhats[k * (N // n_split): (k+1) * (N // n_split), :]
        
        # calculate p(y)
        p_y = np.mean(yhats_, axis=0)
        scores = []
        
        for i in range(yhats_.shape[0]):
            # conditional probability for each image (p(y|x))
            p_yx = yhats_[i, :]
            
            # KL divergence = p(y|x) * (log(p(y|x)) – log(p(y)))
            kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
            kl_d = kl_d.sum(axis=0)
            
            # variation of calculating KL divergence from entropy of conditional & marginal probabilities
            # kl_d = entropy(p_yx, p_y) # formula: -sum(p_i * log(p_i))
            
            scores.append(kl_d)
            
        # collect INCEPTION SCOREs for each group of images
        split_scores.append(np.exp(np.mean(scores)))

    print("\nInception Score: ")
    return np.mean(split_scores), np.std(split_scores)

In [11]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

cifar = dset.CIFAR10(root='data/', download=True,
                         transform=transforms.Compose([
                             transforms.Resize(32),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                         ])
)

Files already downloaded and verified


In [12]:
class IgnoreLabelDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

print("Calculating Inception Score...")
mean_score, std_score = calculate_inception_score(IgnoreLabelDataset(cifar), n_split=10, batch_size=32, resize=True, cuda=True)
print("IS Mean score:",mean_score)
print("IS Std score:",std_score)

Calculating Inception Score...
Loading Inception V3 model...
Predicting class probabilities for images per batch...
Computing the mean kl-div...

Inception Score: 
IS Mean score: 9.672780714173985
IS Std score: 0.14991599423569926


---